 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Utiliser NLPModels pour l'optimisation sans contraintes
Tangi Migot

In this tutorial, we introduce NLPModels for numerical optimization. In particular, we will see how to create a structure providing the derivatives of an optimization problem, and then how we can use Ipopt to solve it.

In this short tutorial, we will use three new packages that belongs to the organization [JuliaSmoothOptimizers](https://juliasmoothoptimizers.github.io/) developed at Polytechnique Montréal.

In [1]:
using Pkg; 
Pkg.activate("nlpmodels") # use the closed environment defined in the nlpmodels folder

  Activating project at `c:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels`


In [3]:
Pkg.add("ADNLPModels")
using ADNLPModels 

Pkg.add("NLPModels")
using NLPModels 

Pkg.add("NLPModelsJuMP")
using NLPModelsJuMP 

Pkg.add("NLPModelsIpopt")
using NLPModelsIpopt 

    Updating registry at `C:\Users\edwar\.julia\registries\General.toml`


   Resolving package versions...


   Installed NLPModelsJuMP ── v0.12.0


   Installed SolverCore ───── v0.3.3
   Installed NLPModelsIpopt ─ v0.10.0


    Updating `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Project.toml`
  [54578032] + ADNLPModels v0.4.0
  [b6b21f68] + Ipopt v1.1.0
  [4076af6c] + JuMP v1.6.0
  [a4795742] + NLPModels v0.19.2
  [f4238b75] + NLPModelsIpopt v0.10.0
  [792afdf1] + NLPModelsJuMP v0.12.0
    Updating `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Manifest.toml`
  [54578032] + ADNLPModels v0.4.0
  [14f7f29c] + AMD v0.5.0
  [6e4b80f9] + BenchmarkTools v1.3.2
  [d360d2e6] + ChainRulesCore v1.15.6
  [9e997f8a] + ChangesOfVariables v0.1.4
  [523fee87] + CodecBzip2 v0.7.2
  [944b1d66] + CodecZlib v0.7.0
  [bbf7d656] + CommonSubexpressions v0.3.0
  [34da2185] + Compat v4.5.0
  [864edb3b] + DataStructures v0.18.13
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.12.2
  [ffbed154] + DocStringExtensions v0.9.3
  [e2ba6199] + ExprTools v0.1.8
  [9aa1b823] + FastClosures v0.3.2
  [f6369f11] + ForwardDiff v0.10.34
  [069b7b12] + FunctionWrappers v1.1.3
  [3587e190] + InverseFunct

...

  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Project.toml`
  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Project.toml`
  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Project.toml`
  No Changes to `C:\Users\edwar\Desktop\H23\MTH8408\MTH8408\lab1\nlpmodels\Manifest.toml`


There exists an online documentation for them:
- NLPModels: [https://juliasmoothoptimizers.github.io/NLPModels.jl/latest/tutorial/](https://juliasmoothoptimizers.github.io/NLPModels.jl/latest/tutorial/)
- NLPModelsJuMP: [https://juliasmoothoptimizers.github.io/NLPModelsJuMP.jl/dev/tutorial/](https://juliasmoothoptimizers.github.io/NLPModelsJuMP.jl/dev/tutorial/)
- NLPModelsIpopt: [https://juliasmoothoptimizers.github.io/NLPModelsIpopt.jl/stable/tutorial/#Tutorial-1](https://juliasmoothoptimizers.github.io/NLPModelsIpopt.jl/stable/tutorial/#Tutorial-1)
- ADNLPModels: [https://juliasmoothoptimizers.github.io/ADNLPModels.jl/latest/tutorial/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/latest/tutorial/)

## NLPModels

What is NLPModels? An NLPModel is a structure allowing us to easily access the derivatives of an optimization problem. There are two principle ways to create such a structure:
i)  Provide the objective function to ADNLPModel, and uses Julia's automatic differentiation;
ii) Convert a problem created in JuMP to an NLPModel.

### 1) Using automatic differentiation

For this purpose, NLPModels has a structure called *ADNLPModel* which is used for unconstrained optimization as follows.

In [4]:
using ADNLPModels #call the package ADNLPModels

#Initialize the objective function, and an initial guess
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2
x0 = [-1.2; 1.0]

#Create an ADNLPModel
nlp = ADNLPModel(f, x0)

ADNLPModel - Model with automatic differentiation backend ADNLPModels.ForwardDiffAD{ForwardDiff.GradientConfig{ForwardDiff.Tag{typeof(f), Float64}, Float64, 2, Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 2}}}}(3, 0, ForwardDiff.GradientConfig{ForwardDiff.Tag{typeof(f), Float64}, Float64, 2, Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 2}}}((Partials(1.0, 0.0), Partials(0.0, 1.0)), ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 2}[Dual{ForwardDiff.Tag{typeof(f), Float64}}(8.06489890091e-312,8.064898901464e-312,8.064898902018e-312), Dual{ForwardDiff.Tag{typeof(f), Float64}}(8.06489890257e-312,8.064898911425e-312,8.06489891198e-312)]))
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0 

In the print, we can already see a number of important information:
- *ADNLPModel - Model with automatic differentiation* as planned Julia uses the automatic differentiation.
- There are a number information relative to the size of the problem. These information are stored in the **meta**.
- There are **Counters**, which are keeping track of the number of evaluations of the objective function, gradient, hessian ...

In [5]:
print(nlp.meta)

  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         



In [6]:
nlp.meta.nvar #returns the size of x

2

In [7]:
print(nlp.counters)

  Counters:
             obj: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 grad: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 cons: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
        cons_lin: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0             cons_nln: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 jcon: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           jgrad: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                  jac: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              jac_lin: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         jac_nln: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                jprod: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0            jprod_lin: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
       jprod_nln: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               jtprod: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0           jtprod_lin: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
      jtprod_nln: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 hess: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                hprod: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           jhess: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               jhprod: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     


In [8]:
neval_obj(nlp) #get the number of evaluation of the objective function

0

To evaluate the objective function and its derivatives, we have at hand the following functions:
- `obj`
- `grad`
- `hess`
- `hprod`
- and more...

In [9]:
x = ones(2)
obj(nlp, x)
grad(nlp, x)
v = rand(2) #vector of size 2 with random numbers between 0 and 1
hprod(nlp, x, v)

2-element Vector{Float64}:
  367.4400538519188
 -182.95120030496855

It is very important to note here that the function `hess` returns only the lower triangular of the hessian matrix, since it is always a symmetric matrix and it uses less memory this way.

In [10]:
hess(nlp, x)

2×2 LinearAlgebra.Symmetric{Float64, Matrix{Float64}}:
  802.0  -400.0
 -400.0   200.0

In [11]:
using LinearAlgebra

In [12]:
hprod(nlp, x, v) - Symmetric(hess(nlp, x), :L) * v

2-element Vector{Float64}:
 -5.684341886080802e-14
  0.0

### 2) Use a JuMP model

For this purpose, we use a package associated to `NLPModels`, which is called `NLPModelsJuMP`.

In [13]:
using NLPModels, NLPModelsJuMP, JuMP

x0 = [-1.2; 1.0]
model = Model() # No solver is required
@variable(model, x[i=1:2], start=x0[i])
@NLobjective(model, Min, (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2)

nlp_jump = MathOptNLPModel(model)

MathOptNLPModel
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         

  Counters:
             obj: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅

To access the objective function and the derivatives, we proceed the exact same way as for `ADNLPModel` before.

In [ ]:
x = ones(2)
obj(nlp_jump, x)
grad(nlp_jump, x)
v = rand(2)
hprod(nlp_jump, x, v)
hess(nlp_jump, x) #sparse matrix

## Solve an NLPModel with Ipopt

The problem created with `JuMP` can be solved with `Ipopt`.

In [14]:
using JuMP, Ipopt

model = Model(Ipopt.Optimizer)
x0 = [-1.2; 1.0]
@variable(model, x[i=1:2], start=x0[i])
@NLobjective(model, Min, (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2)
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

The same can be achieved very easily with `NLPModels` using the package `NLPModelsIpopt` and its function `ipopt`.

In [15]:
using NLPModels, NLPModelsIpopt

nlp = ADNLPModel(x -> (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2, [-1.2; 1.0])
stats = ipopt(nlp)
print(stats)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.4200000e+01 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

:Solve_Succeeded
